In [ ]:
#select with baseline to use for the z-scoring
#not a part of the main analysis

In [ ]:
import preprocessing.data_preprocessing.refractor_utils as ru
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from pathlib import Path

import utils.helper_func as hf
import utils.preprocessing as preprocessing
import utils.plotting as plotting

In [3]:
top_dir = '/home/anastasia/epiphyte/anastasia/output'
folder = '05-Hilbert_transform'

patient_id = 46
session_nr = 1

df_patient_info = pd.read_csv(f'{top_dir}/{patient_id}_channel_info.csv')

fs = 32768
#The resulting sample rate is up / down times the original sample rate.
up = 1
down=32
fs_downs = (up/down)*fs
dt = 1/fs_downs      # sampling period/time/interval or time resolution, often denoted as T

signal_len = np.int(fs_downs*1.5)+1
times = np.linspace(-0.5, 1, signal_len)
times = times*1000
time_zero_idx = np.where(times == hf.find_nearest(times, 0))[0][0]

df_stim_info = pd.read_csv(f'{top_dir}/{patient_id}_df_stim_info.csv')

all_stim = np.unique(df_stim_info['stim_id'])
all_stim_name = np.unique(df_stim_info['stim_name'])
all_brain_areas = np.unique(df_patient_info['recording_site'])

#filtering_types = ['slow_gamma']
filtering_types = ['theta', 'slow_gamma', 'fast_gamma']

In [59]:
x_pos = np.arange(len(filtering_types))

for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    baseline_500_pre = []
    baseline_100_pre = []
    baseline_500_post = []
    baseline_100_post = []
    std_500_pre = []
    std_100_pre = []
    std_500_post = []
    std_100_post = []
    
    for filtering_type in filtering_types:
        bsl_500_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_500_pre.npy')
        bsl_100_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_100_pre.npy')
        bsl_500_post = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_500_post.npy')
        bsl_100_post = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_100_post.npy')
        
        baseline_500_pre.append(bsl_500_pre)
        baseline_100_pre.append(bsl_100_pre)
        baseline_500_post.append(bsl_500_post)
        baseline_100_post.append(bsl_100_post)
        
        std_500_pre.append(np.std(bsl_500_pre))
        std_100_pre.append(np.std(bsl_100_pre))
        std_500_post.append(np.std(bsl_500_post))
        std_100_post.append(np.std(bsl_100_post))
    
    #baselines_pre = [aluminum_mean, copper_mean, steel_mean]
    #stds_pre = [aluminum_std, copper_std, steel_std]
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharey=True, figsize=(20, 5))
    title = f'Baseline, channel {ch}, recording_site {ch_site}'
    #file_name = f"{{'amplitude_envelope_and_mean_signal_channel'!'{ch}', 'recording_site'!'{ch_site}', 'stim_index'!'{current_stim_index}', 'stim_name'!'{current_stim_name}'}}"
    fig.suptitle(title)

    ax1.bar(x_pos, np.mean(baseline_500_pre,axis=1), yerr=std_500_pre, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(filtering_types)
    ax1.set_title('Baseline pre-movie 500 ms')
    ax1.set_ylabel('log10 power')
    ax2.bar(x_pos, np.mean(baseline_100_pre,axis=1), yerr=std_100_pre, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(filtering_types)
    ax2.set_title('Baseline pre-movie 100 ms')
    ax3.bar(x_pos, np.mean(baseline_500_pre,axis=1), yerr=std_500_post, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(filtering_types)
    ax3.set_title('Baseline post-movie 500 ms')
    ax4.bar(x_pos, np.mean(baseline_100_post,axis=1), yerr=std_100_post, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels(filtering_types)
    ax4.set_title('Baseline post-movie 100 ms')
    
    #plt.show()
    fig.savefig(f'{top_dir}/{folder}/plots/baseline/{title}', facecolor='white', transparent=False)
    plt.close()

    
        


In [69]:
wilcoxon_post

[WilcoxonResult(statistic=286.0, pvalue=0.03851265981574622),
 WilcoxonResult(statistic=326.0, pvalue=0.11659906220946456),
 WilcoxonResult(statistic=448.0, pvalue=0.9650934616100189)]

In [71]:
x_pos = np.arange(4)
wilcoxon_pre_all = []
wilcoxon_post_all = []


for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    print(ch)
    ch_site = df_patient_info.loc[i,'recording_site']
    
    baseline_500_pre = []
    baseline_100_pre = []
    baseline_500_post = []
    baseline_100_post = []
    std_500_pre = []
    std_100_pre = []
    std_500_post = []
    std_100_post = []
    baseline = []
    stds = []
    wilcoxon_pre = []
    wilcoxon_post = []
    
    for filtering_type in filtering_types:
        print(filtering_type)
        bsl_500_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_500_pre.npy')
        bsl_100_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_100_pre.npy')
        _, wilc_pre = scipy.stats.wilcoxon(bsl_500_pre, bsl_100_pre)
        print(wilc_pre)
        wilcoxon_pre.append(wilc_pre)
        bsl_500_post = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_500_post.npy')
        bsl_100_post = np.load(f'{top_dir}/{folder}/{filtering_type}/baseline/{ch}_{ch_site}_mean_baseline_100_post.npy')
        _, wilc_post = scipy.stats.wilcoxon(bsl_500_post, bsl_100_post)
        print(wilc_post)
        wilcoxon_post.append(wilc_post)
        
        baseline.append(bsl_500_pre)
        baseline.append(bsl_100_pre)
        baseline.append(bsl_500_post)
        baseline.append(bsl_100_post)
        
        stds.append(np.std(bsl_500_pre))
        stds.append(np.std(bsl_100_pre))
        stds.append(np.std(bsl_500_post))
        stds.append(np.std(bsl_100_post))
    
    baselines_theta = baseline[0:4]
    baselines_slow_gamma = baseline[4:8]
    baselines_fast_gamma = baseline[8:12]
    std_theta = stds[0:4]
    std_slow_gamma = stds[4:8]
    std_fast_gamma = stds[8:12]
    
    wilcoxon_pre_all.append(wilcoxon_pre)
    wilcoxon_post_all.append(wilcoxon_post)
    
    xlabels = ['pre 500 ms', 'pre 100 ms', 'post 500 ms', 'post 100 ms', ]
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(16, 5))
    title = f'Baseline, channel {ch}, recording_site {ch_site}'
    #file_name = f"{{'amplitude_envelope_and_mean_signal_channel'!'{ch}', 'recording_site'!'{ch_site}', 'stim_index'!'{current_stim_index}', 'stim_name'!'{current_stim_name}'}}"
    fig.suptitle(title)
    
    ax1.bar(x_pos, np.mean(baselines_theta,axis=1), yerr=std_theta, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels(xlabels)
    ax1.set_title(f'theta, wilcoxon_pre = {wilcoxon_pre[0]}, wilcoxon_post = {wilcoxon_post[0]}')
    ax1.set_ylabel('log10 power')
    ax2.bar(x_pos, np.mean(baselines_slow_gamma,axis=1), yerr=std_slow_gamma, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels(xlabels)
    ax2.set_title('slow gamma')
    ax3.bar(x_pos, np.mean(baselines_fast_gamma,axis=1), yerr=std_fast_gamma, align='center', alpha=0.5, ecolor='black', capsize=10)
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels(xlabels)
    ax3.set_title('fast gamma')

    #plt.show()
    fig.savefig(f'{top_dir}/{folder}/plots/baseline_per_freq/{title}', facecolor='white', transparent=False)
    plt.close()

    
        


CSC1
theta
0.21809447610564103
0.915359293647618
slow_gamma
0.6844714416681018
0.17095202371801554
fast_gamma
0.384844599148347
0.3388043965699914
CSC2
theta
0.915359293647618
0.5779285296327135
slow_gamma
0.3141535102447033
0.18297845295861048
fast_gamma
0.3141535102447033
0.9054458470326758
CSC3
theta
0.9252859228635083
0.5115395522282624
slow_gamma
0.3325282563417864
0.18712347510056748
fast_gamma
0.3202027778610119
0.675309144438951
CSC4
theta
0.4127921340587756
0.5115395522282624
slow_gamma
0.8856646154509445
0.1318897569057361
fast_gamma
0.37131549425571186
0.4493664606726153
CSC5
theta
0.8463271720076432
0.5525629281728892
slow_gamma
0.5115395522282624
0.22758485936587003
fast_gamma
0.1318897569057361
0.778455435966652
CSC6
theta
0.8955471131969037
0.5694126720967009
slow_gamma
0.4272055341199906
0.30228191596887943
fast_gamma
0.4056948211445802
0.6936804113311528
CSC7
theta
0.836548415131769
0.5035299195137813
slow_gamma
0.39172073941517005
0.07684909152831935
fast_gamma
0.8659

In [110]:
a = np.array(wilcoxon_post_all)
b = np.argwhere(a <= 0.05)
len(b)

16

In [106]:
np.arange(0,80,10)

array([ 0, 10, 20, 30, 40, 50, 60, 70])

In [113]:
import statsmodels
for i in np.arange(0,80,10):
    c = statsmodels.stats.multitest.multipletests(a[:i+10,2], alpha=0.05, method='simes-hochberg')[1]
    b = np.argwhere(c <= 0.05)
    print(len(b))

0
0
0
0
0
1
1
0


In [94]:
c

array([0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501179,
       0.99501179, 0.99501179, 0.99501179, 0.99501179, 0.99501

In [88]:
a

array([[9.15359294e-01, 1.70952024e-01, 3.38804397e-01],
       [5.77928530e-01, 1.82978453e-01, 9.05445847e-01],
       [5.11539552e-01, 1.87123475e-01, 6.75309144e-01],
       [5.11539552e-01, 1.31889757e-01, 4.49366461e-01],
       [5.52562928e-01, 2.27584859e-01, 7.78455436e-01],
       [5.69412672e-01, 3.02281916e-01, 6.93680411e-01],
       [5.03529920e-01, 7.68490915e-02, 9.35224198e-01],
       [4.27205534e-01, 1.70952024e-01, 4.64495586e-01],
       [6.93680411e-01, 4.61155263e-02, 3.71315494e-01],
       [8.17067936e-01, 1.50172581e-02, 7.89579153e-02],
       [1.59527589e-01, 9.50700028e-02, 9.15359294e-01],
       [3.58085486e-01, 7.21575162e-01, 3.91720739e-01],
       [4.79903984e-01, 5.11539552e-01, 4.56895840e-01],
       [3.64663001e-01, 2.42366342e-01, 5.65453621e-02],
       [2.04397602e-01, 9.55129516e-01, 6.48114066e-01],
       [9.15359294e-01, 3.97003987e-02, 4.79903984e-01],
       [7.49845247e-01, 1.82978453e-01, 2.18094476e-01],
       [7.21575162e-01, 1.67077